# Data-Level Imbalance Handling Techniques: A Comparative Study on Credit Card Fraud Detection
DSCI Senior Project  
Malak Gaballa - Masa Tantawy - Moustafa El Mahdy
# Preprocessing Phase

## Imports and Dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
from google.colab import drive
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as kr
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas

#od.download("https://www.kaggle.com/datasets/nelgiriyewithana/credit-card-fraud-detection-dataset-2023")
od.download("https://www.kaggle.com/datasets/kelvinkelue/credit-card-fraud-prediction/data")

Skipping, found downloaded files in "./credit-card-fraud-prediction" (use force=True to force download)


In [ ]:
import os

# Define the directory path where the dataset should be
directory_path = "./credit-card-fraud-prediction"

# List all files in the directory
files = os.listdir(directory_path)
print(files)

['fraud test.csv']


In [ ]:
import os

# Define the directory path
directory_path = "./credit-card-fraud-prediction"

# List all files in the directory
files = os.listdir(directory_path)
print(files)


['fraud test.csv']


In [ ]:
# Assuming the dataset is in the default location where opendatasets saves files
dataset_path = "./credit-card-fraud-prediction/fraud test.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(dataset_path)

# Display the first few rows of the DataFrame to confirm it is loaded correctly
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,21/06/2020 12:14,2.291160e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,19/03/1968,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,21/06/2020 12:14,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",17/01/1990,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,21/06/2020 12:14,3.598220e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",21/10/1970,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,21/06/2020 12:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,25/07/1987,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,21/06/2020 12:15,3.526830e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,06/07/1955,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             555719 non-null  int64  
 1   trans_date_trans_time  555719 non-null  object 
 2   cc_num                 555719 non-null  float64
 3   merchant               555719 non-null  object 
 4   category               555719 non-null  object 
 5   amt                    555719 non-null  float64
 6   first                  555719 non-null  object 
 7   last                   555719 non-null  object 
 8   gender                 555719 non-null  object 
 9   street                 555719 non-null  object 
 10  city                   555719 non-null  object 
 11  state                  555719 non-null  object 
 12  zip                    555719 non-null  int64  
 13  lat                    555719 non-null  float64
 14  long                   555719 non-nu

In [ ]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

## Data preparation

In [ ]:
df.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [ ]:
df.drop(['Unnamed: 0','cc_num', 'first', 'last', 'zip', 'trans_num','unix_time'], axis= 1, inplace=True)
df.head()

,trans_date_trans_time,merchant,category,amt,gender,street,city,state,lat,long,city_pop,job,dob,merch_lat,merch_long,is_fraud
0,21/06/2020 12:14,fraud_Kirlin and Sons,personal_care,2.86,M,351 Darlene Green,Columbia,SC,33.9659,-80.9355,333497,Mechanical engineer,19/03/1968,33.986391,-81.200714,0
1,21/06/2020 12:14,fraud_Sporer-Keebler,personal_care,29.84,F,3638 Marsh Union,Altonah,UT,40.3207,-110.4360,302,"Sales professional, IT",17/01/1990,39.450498,-109.960431,0
2,21/06/2020 12:14,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,F,9333 Valentine Point,Bellmore,NY,40.6729,-73.5365,34496,"Librarian, public",21/10/1970,40.495810,-74.196111,0
3,21/06/2020 12:15,fraud_Haley Group,misc_pos,60.05,M,32941 Krystal Mill Apt. 552,Titusville,FL,28.5697,-80.8191,54767,Set designer,25/07/1987,28.812398,-80.883061,0
4,21/06/2020 12:15,fraud_Johnston-Casper,travel,3.19,M,5783 Evan Roads Apt. 465,Falmouth,MI,44.2529,-85.0170,1126,Furniture designer,06/07/1955,44.959148,-85.884734,0


In [ ]:
df.shape

(555719, 16)

In [ ]:
print('Class 0: ',sum(df.is_fraud == 0))
print('Class 1: ',sum(df.is_fraud == 1))

Class 0:  553574
Class 1:  2145


In [ ]:
# Convert 'trans_date_trans_time' to datetime
df['dob'] = pd.to_datetime(df['dob'])
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

# Extract Year, Day, Month, and Hour
df['Year'] = df['trans_date_trans_time'].dt.year
df['Day'] = df['trans_date_trans_time'].dt.day
df['Month'] = df['trans_date_trans_time'].dt.month
df['Hour'] = df['trans_date_trans_time'].dt.hour

df['age'] = (df['trans_date_trans_time'] - df['dob']).dt.days // 365
df.drop(columns='trans_date_trans_time', inplace=True)
df.drop(columns='dob', inplace=True)

<ipython-input-16-31926d4f97a2>:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['dob'] = pd.to_datetime(df['dob'])
<ipython-input-16-31926d4f97a2>:3: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])


In [ ]:
df.head()

,merchant,category,amt,gender,street,city,state,lat,long,city_pop,job,merch_lat,merch_long,is_fraud,Year,Day,Month,Hour,age
0,fraud_Kirlin and Sons,personal_care,2.86,M,351 Darlene Green,Columbia,SC,33.9659,-80.9355,333497,Mechanical engineer,33.986391,-81.200714,0,2020,21,6,12,52
1,fraud_Sporer-Keebler,personal_care,29.84,F,3638 Marsh Union,Altonah,UT,40.3207,-110.4360,302,"Sales professional, IT",39.450498,-109.960431,0,2020,21,6,12,30
2,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,F,9333 Valentine Point,Bellmore,NY,40.6729,-73.5365,34496,"Librarian, public",40.495810,-74.196111,0,2020,21,6,12,49
3,fraud_Haley Group,misc_pos,60.05,M,32941 Krystal Mill Apt. 552,Titusville,FL,28.5697,-80.8191,54767,Set designer,28.812398,-80.883061,0,2020,21,6,12,32
4,fraud_Johnston-Casper,travel,3.19,M,5783 Evan Roads Apt. 465,Falmouth,MI,44.2529,-85.0170,1126,Furniture designer,44.959148,-85.884734,0,2020,21,6,12,65


In [ ]:
df["gender"] = df["gender"].replace({'M': 0, 'F': 1})

In [ ]:
df.nunique()

merchant         693
category          14
amt            37256
gender             2
street           924
city             849
state             50
lat              910
long             910
city_pop         835
job              478
merch_lat     546490
merch_long    551770
is_fraud           2
Year               1
Day               31
Month              7
Hour              24
age               82
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   merchant    555719 non-null  object 
 1   category    555719 non-null  object 
 2   amt         555719 non-null  float64
 3   gender      555719 non-null  int64  
 4   street      555719 non-null  object 
 5   city        555719 non-null  object 
 6   state       555719 non-null  object 
 7   lat         555719 non-null  float64
 8   long        555719 non-null  float64
 9   city_pop    555719 non-null  int64  
 10  job         555719 non-null  object 
 11  merch_lat   555719 non-null  float64
 12  merch_long  555719 non-null  float64
 13  is_fraud    555719 non-null  int64  
 14  Year        555719 non-null  int32  
 15  Day         555719 non-null  int32  
 16  Month       555719 non-null  int32  
 17  Hour        555719 non-null  int32  
 18  age         555719 non-null  int64  
dtypes:

In [ ]:
df['location'] = df['city'] + ', ' + df['state']
df.drop(['city', 'state','street'], axis=1,inplace=True)

In [ ]:
df['lat_long'] = df['lat'].astype(str) + ', ' + df['long'].astype(str)
df.drop(['lat', 'long'], axis=1, inplace=True)

In [ ]:
df['merchant_lat_long'] = df['merch_lat'].astype(str) + ', ' + df['merch_long'].astype(str)
df.drop(['merch_lat', 'merch_long'], axis=1,inplace=True)

In [ ]:
df.head()

,merchant,category,amt,gender,city_pop,job,is_fraud,Year,Day,Month,Hour,age,location,lat_long,merchant_lat_long
0,fraud_Kirlin and Sons,personal_care,2.86,0,333497,Mechanical engineer,0,2020,21,6,12,52,"Columbia, SC","33.9659, -80.9355","33.986391, -81.200714"
1,fraud_Sporer-Keebler,personal_care,29.84,1,302,"Sales professional, IT",0,2020,21,6,12,30,"Altonah, UT","40.3207, -110.436","39.450498, -109.960431"
2,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,1,34496,"Librarian, public",0,2020,21,6,12,49,"Bellmore, NY","40.6729, -73.5365","40.49581, -74.196111"
3,fraud_Haley Group,misc_pos,60.05,0,54767,Set designer,0,2020,21,6,12,32,"Titusville, FL","28.5697, -80.8191","28.812398, -80.883061"
4,fraud_Johnston-Casper,travel,3.19,0,1126,Furniture designer,0,2020,21,6,12,65,"Falmouth, MI","44.2529, -85.017","44.959148, -85.884734"


In [ ]:
df['job'].nunique()

478

In [ ]:
engineering_technical_jobs = ['Exhibition designer', 'Surveyor, land/geomatics', 'Naval architect',
                              'Materials engineer', 'Environmental consultant', 'Systems developer',
                              'IT trainer', 'Audiological scientist', 'Podiatrist', 'Mining engineer',
                              'Water engineer', 'Quantity surveyor', 'Mechanical engineer', 'Engineer, automotive',
                              'Petroleum engineer', 'Chemical engineer', 'Engineer, biomedical',
                              'Drilling engineer', 'Electrical engineer', 'Cytogeneticist',
                              'Engineer, control and instrumentation', 'Technical brewer',
                              'Engineering geologist', 'Designer, industrial/product',
                             'Pilot, airline', 'Town planner', 'Radiographer, therapeutic',
                              'Applications developer', 'Systems analyst', 'Structural engineer',
                              'Engineer, mining', 'Physicist, medical', 'Engineer, electronics',
                              'Site engineer', 'Manufacturing engineer', 'Designer, exhibition/display',
                              'Electronics engineer', 'Building control surveyor', 'Wellsite geologist',
                              'Maintenance engineer', 'Product designer','Risk analyst',
                              'Occupational psychologist', 'Colour technologist',
                              'Research officer, political party', 'Insurance risk surveyor',
                              'Operations geologist', 'Geologist, engineering', 'Energy manager',
                              'Engineer, petroleum', 'Operational researcher',
                              'Accountant, chartered public finance', 'Database administrator',
                              'Communications engineer', 'Seismic interpreter', 'Civil engineer, contracting',
                              'Geologist, wellsite', 'Engineer, technical sales', 'Biomedical engineer',
                              'Engineer, drilling', 'Aeronautical engineer', 'Network engineer',
                              'Health and safety adviser','Statistician', 'Transport planner',
                              'Scientific laboratory technician', 'Cartographer',
                              'Telecommunications researcher', 'Hydrologist',
                              'Emergency planning/management officer','Engineer, maintenance',
                              'Production engineer', 'Engineer, civil (contracting)', 'Engineer, agricultural',
                              'Engineer, communications', 'Rural practice surveyor',
                              'Geophysicist/field seismologist', 'Control and instrumentation engineer',
                              'Engineer, building services', 'Engineer, aeronautical', 'Hydrographic surveyor',
                              'Architectural technologist', 'Programmer, applications',
                              'Engineer, broadcasting (operations)','Armed forces logistics/support/administrative officer',
                              'Field seismologist', 'Environmental health practitioner', 'Physiological scientist',
                              'Building services engineer', 'Surveyor, mining','Research scientist (maths)', 'Loss adjuster, chartered',
                              'Insurance claims handler','Horticultural consultant', 'Geneticist, molecular', 'Clinical cytogeneticist',
                              'Minerals surveyor', 'Field trials officer', 'Hydrogeologist', 'Engineer, structural', 'Broadcast engineer',
                              'Analytical chemist', 'Surveyor, hydrographic', 'Engineer, civil (consulting)','Engineer, materials',
                              'Air traffic controller', 'Armed forces technical officer', 'Contracting civil engineer',
                              'Building surveyor','Engineer, land','Engineer, production','Metallurgist',
                              'Manufacturing systems engineer','Planning and development surveyor','Engineer, water',
                              'Software engineer','Engineer, site',
                             'Geochemist','Programmer, multimedia','Intelligence analyst',
                              'Information systems manager','Environmental manager','Land surveyor',
                              'Animal technologist','Arboriculturist','Research scientist (physical sciences)',
                              'Research scientist (life sciences)','Dispensing optician','Chemist, analytical',
                              'Engineer, manufacturing','Information officer', 'Chief Technology Officer', ]


healthcare_jobs = ['Paramedic', 'Child psychotherapist', 'Radiographer, therapeutic', 'Exercise physiologist',
                   'Hospital doctor', 'Forensic psychologist', 'Learning disability nurse',
                   'Psychologist, counselling', "Nurse, children's", 'Paediatric nurse', 'Mental health nurse'
                   'Audiological scientist', 'Psychologist, forensic', 'Counsellor', 'Firefighter',
                   'Physiotherapist', 'Toxicologist', 'Senior tax professional/tax inspector', 'Physician',
                   'Therapist, occupational','Health physicist','Optician, dispensing', 'Health service manager', 'Osteopath', 'Doctor, general practice', 'Doctor, hospital', 'Therapist, sports', 'Music therapist', 'Neurosurgeon', 'General practice doctor', 'Medical secretary', 'Nutritional therapist', 'Clinical biochemist', 'Pharmacist, community',
                   'Health promotion specialist', 'Psychotherapist', 'Biomedical engineer', 'Optometrist',
                  'Community pharmacist', 'Psychiatric nurse', 'Diagnostic radiographer', 'Acupuncturist', 'Orthoptist', 'Physiological scientist', 'Pharmacist, hospital', 'Medical technical officer',
                   'Hospital pharmacist', 'Pathologist', 'Nurse, mental health', 'Clinical psychologist','Ambulance person',
                  'Oncologist', 'Occupational therapist', 'Veterinary surgeon', 'Homeopath',
                  'Counselling psychologist','Scientist, biomedical','Clinical research associate',
                   'Scientist, audiological','Chiropodist','Biomedical scientist','Psychotherapist, child',
                   'Occupational hygienist','Immunologist','Health visitor','Radiographer, diagnostic','Psychiatrist',
                   'Mental health nurse','Pharmacologist', 'Medical sales representative','Scientist, physiological',
                   'Phytotherapist','Medical physicist','Sport and exercise psychologist','Psychologist, sport and exercise',
                   'Embryologist, clinical', 'Research scientist (medical)', 'Scientist, research (medical)',
                   'Scientist, clinical (histocompatibility and immunogenetics)', 'Scientist, research (physical sciences)']

creative_design_jobs = ['Exhibition designer', 'Designer, ceramics/pottery', 'Musician',
                        'Magazine features editor', 'Clothing/textile technologist', 'Fine artist',
                        'Ceramics designer','Press photographer', 'Exhibitions officer, museum/gallery',
                        'Broadcast presenter', 'Private music teacher', 'Editor, commissioning',
                        'Designer, jewellery', 'Video editor', 'Designer, furniture', 'Press sub',
                        'Product designer','Community arts worker', 'Therapist, art', 'Glass blower/designer', 'Architect', 'Landscape architect', 'Furniture designer', 'Editor, film/video',
                        'Herpetologist', 'Art therapist', 'Set designer', 'Therapist, drama','Theatre director', 'Special effects artist', 'Animator', 'Designer, interior/spatial', 'Illustrator', 'Location manager', 'Jewellery designer', 'Presenter, broadcasting', 'Designer, multimedia', 'Biochemist, clinical', 'Museum education officer',
                        'Environmental education officer', 'Art gallery manager', 'Stage manager', 'Furniture conservator/restorer',
                       'Magazine journalist', 'Conservator, museum/gallery', 'Interior and spatial designer',
                        'Designer, television/film set', 'Artist', 'Textile designer', 'Broadcast journalist',
                        'Museum/gallery exhibitions officer','Film/video editor','Web designer','Music tutor',
                        'Archivist','Advertising copywriter','Designer, textile','Theatre manager',
                        'Dance movement psychotherapist','Visual merchandiser','Museum/gallery conservator',
                        'Industrial/product designer','Dancer', 'Copywriter', 'Bookseller',  'Scientist, marine']

legal_jobs = ['Barrister', 'Trade mark attorney', 'Chartered loss adjuster', 'Probation officer',
              "Barrister's clerk", 'Licensed conveyancer', 'Patent attorney','Tax adviser',
             'Chartered accountant', 'Chartered legal executive (England and Wales)', 'Lawyer',
             'Solicitor, Scotland', 'Solicitor', 'Legal secretary', 'Tax inspector','Chief Executive Officer',
              'Chartered public finance accountant','Comptroller','Investment banker, corporate',
              'Trading standards officer','Company secretary','Investment banker, operational',
              'Operational investment banker','Ship broker', "Insurance underwriter"]

education_jobs = ['Librarian, public', 'Secondary school teacher', 'Early years teacher',
                  'Special educational needs teacher', 'Further education lecturer',
                 'English as a second language teacher', 'Lecturer, further education', 'Education officer, museum',
                  'Lecturer, higher education',
                  'Development worker, international aid', 'Teacher, early years/pre', 'Administrator, education',
                 'Teacher, secondary school', 'Higher education careers adviser',
                 'Teacher, primary school', 'Teaching laboratory technician', 'Learning mentor',
                  'Teacher, English as a foreign language', 'Education administrator', 'Academic librarian', 'TEFL teacher',
                 'English as a foreign language teacher', 'Primary school teacher',
                  'Teacher, adult education', 'Associate Professor', 'Professor Emeritus', 'Careers adviser',
                  'Educational psychologist','Librarian, academic','Animal nutritionist','Public house manager',
                  'Training and development officer','Teacher, special educational needs','Science writer',
                  'Records manager','Catering manager','Sports administrator',
                  "Armed forces training and education officer", 'Scientist, research (maths)',  'Archaeologist']

business_jobs = ['Financial adviser', 'Sales professional, IT', 'Pensions consultant',
                 'Dealer', 'Senior tax professional/tax inspector', 'Futures trader',
                 'Advertising account planner', 'Chief Strategy Officer',
                'Facilities manager', 'Public relations account executive',
                 'Programme researcher, broadcasting/film/video', 'Call centre manager',
                 'Retail merchandiser', 'Insurance broker', 'Chief Operating Officer',
                 'Careers information officer', 'Regulatory affairs officer',
                 'Tourist information centre manager', 'Chief Financial Officer',
                 'Administrator, charities/voluntary organisations','Market researcher', 'Investment analyst', 'Media buyer', 'Public affairs consultant', 'Charity fundraiser', 'Chief Marketing Officer', 'Retail buyer', 'Corporate investment banker',
                 'Advertising account executive', 'Retail banker', 'Pension scheme manager',
                'Sales executive', 'Management consultant', 'Logistics and distribution manager',
                 'Equities trader', 'Chief of Staff', 'Product manager', 'IT consultant', 'Cabin crew',
                'Accountant, chartered certified', 'Charity officer', 'Data scientist', 'Restaurant manager, fast food', 'Civil Service administrator', 'Tour manager', 'Purchasing manager', 'Marketing executive', 'Merchandiser, retail', 'Local government officer',
                 'Industrial buyer', 'Sales promotion account executive', 'Personnel officer', 'Warehouse manager',
                 'Buyer, industrial','Leisure centre manager','Travel agency manager','Accounting technician',
                 'Secretary/administrator','Buyer, retail','Human resources officer','Retail manager','Economist',
                "Production manager","Contractor","Financial trader","Public relations officer","Air broker",
                 "Hotel manager","Accountant, chartered", 'Administrator', 'Commercial/residential surveyor',
                'Fitness centre manager',  'Product/process development scientist']


media_jobs = ['Copywriter, advertising', 'Television production assistant', 'Multimedia programmer',
              'Radio broadcast assistant', 'Social researcher', 'Radio producer',
              'Television/film/video producer', 'Lexicographer','Press photographer',
              'Broadcast presenter', 'Producer, radio', 'Camera operator', 'Press sub', 'Video editor',
             'Media buyer', 'Editor, film/video', 'Editor, magazine features',
             'Commissioning editor', 'Media planner', 'Presenter, broadcasting', 'Production assistant, radio',
             'Television camera operator', 'Writer','Production assistant, television',
              'Producer, television/film/video', 'Broadcast journalist', 'Journalist, newspaper','Gaffer',
              'Television floor manager','Science writer',' Therapist, music','Event organiser', "Public librarian"]

agriculture_jobs = ['Agricultural consultant', 'Land/geomatics surveyor', 'Geoscientist',
                    'Energy engineer', 'Quantity surveyor', 'Surveyor, minerals',
                   'Fisheries officer', 'Amenity horticulturist', 'Commercial horticulturist',
                    'Tree surgeon', 'Warden/ranger', 'Therapist, horticultural', 'Soil scientist','Waste management officer',
                    'Surveyor, rural practice', 'Farm manager', 'Horticulturist, commercial',
                   'Quarry manager', 'Plant breeder/geneticist','Nature conservation officer',
                    'Conservator, furniture', 'Forest/woodland manager', 'Estate manager/land agent',
                    'Animal nutritionist',' Arboriculturist', 'Mudlogger', 'Garment/textile technologist',
                    'Water quality scientist',  'Land', 'Herbalist']


social_services_jobs = ['Claims inspector/assessor', 'Race relations officer', 'Development worker, community',
                        'Probation officer', 'Surgeon', 'Social researcher', 'Curator',
                        'Research officer, trade union', 'Learning disability nurse',
                        'Psychologist, counselling', 'Development worker, international aid',
                        'Equality and diversity officer', 'Administrator, charities/voluntary organisations',
                        'Police officer','Advice worker', 'Horticultural therapist', 'Immigration officer',
                        'Arts development officer', 'Aid worker', 'Volunteer coordinator',
                       'Psychologist, clinical', 'Sub', 'Copy',  'Make','Social research officer, government',
                        'Outdoor activities/education manager', 'Community development worker',
                       'Education officer, community','Prison officer','Social services jobs','Community education officer',
                        'Heritage manager','Health visitor','Mental health nurse','Dance movement psychotherapist',
                        'Therapist, music','Ecologist','Historic buildings inspector/conservation officer',
                        'Conservation officer, historic buildings','Tourism officer','Sports development officer',
                        'Civil Service fast streamer','Interpreter','Oceanographer','Administrator, local government',
                        'Administrator, arts','Theme park manager','Barista','Air cabin crew',"Politician's assistant",
                        'Freight forwarder',  'Airline pilot']

#all_jobs= engineering_technical_jobs+social_services_jobs+ healthcare_jobs+creative_design_jobs+legal_jobs+education_jobs+business_jobs+media_jobs+ agriculture_jobs
#len(all_jobs)

In [ ]:
# Creating a dictionary to map the jobs to their industry
gr1 = {legal_jobs[i]: 'Legal'  for i in range(len(legal_jobs))}
gr2 = {creative_design_jobs[i]: 'Creative'  for i in range(len(creative_design_jobs))}
gr3 = {agriculture_jobs[i]: 'Agriculture'  for i in range(len(agriculture_jobs))}
gr4 = {media_jobs[i]: 'Media'  for i in range(len(media_jobs))}
gr5 = {education_jobs[i]: 'Education'  for i in range(len(education_jobs))}
gr6 = {business_jobs[i]: 'Business'  for i in range(len(business_jobs))}
gr7 = {social_services_jobs[i]: 'Social_Services'  for i in range(len(social_services_jobs))}
gr8 = {healthcare_jobs[i]: 'Healthcare'  for i in range(len(healthcare_jobs))}
gr9 = {engineering_technical_jobs[i]: 'Engineering_Technical'  for i in range(len(engineering_technical_jobs))}

merged_dict = {**gr1, **gr2, **gr3, **gr4, **gr5, **gr6, **gr7, **gr8, **gr9 }

In [ ]:
df= df.replace({"job": merged_dict})
df_job= pd.get_dummies(df.job)
df_job.rename(columns=lambda x: 'job_'+ x, inplace = True)
df=df.join(df_job)
df.drop("job", axis=1, inplace=True)

In [ ]:
df.head()

,merchant,category,amt,gender,city_pop,is_fraud,Year,Day,Month,Hour,...,merchant_lat_long,job_Agriculture,job_Business,job_Creative,job_Education,job_Engineering_Technical,job_Healthcare,job_Legal,job_Media,job_Social_Services
0,fraud_Kirlin and Sons,personal_care,2.86,0,333497,0,2020,21,6,12,...,"33.986391, -81.200714",False,False,False,False,True,False,False,False,False
1,fraud_Sporer-Keebler,personal_care,29.84,1,302,0,2020,21,6,12,...,"39.450498, -109.960431",False,True,False,False,False,False,False,False,False
2,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,1,34496,0,2020,21,6,12,...,"40.49581, -74.196111",False,False,False,True,False,False,False,False,False
3,fraud_Haley Group,misc_pos,60.05,0,54767,0,2020,21,6,12,...,"28.812398, -80.883061",False,False,True,False,False,False,False,False,False
4,fraud_Johnston-Casper,travel,3.19,0,1126,0,2020,21,6,12,...,"44.959148, -85.884734",False,False,True,False,False,False,False,False,False


In [ ]:
df['category'].nunique()

14

In [ ]:
df = pd.get_dummies(df, columns=['category'])
df.head()

,merchant,amt,gender,city_pop,is_fraud,Year,Day,Month,Hour,age,...,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,fraud_Kirlin and Sons,2.86,0,333497,0,2020,21,6,12,52,...,False,False,False,False,False,False,True,False,False,False
1,fraud_Sporer-Keebler,29.84,1,302,0,2020,21,6,12,30,...,False,False,False,False,False,False,True,False,False,False
2,"fraud_Swaniawski, Nitzsche and Welch",41.28,1,34496,0,2020,21,6,12,49,...,False,True,False,False,False,False,False,False,False,False
3,fraud_Haley Group,60.05,0,54767,0,2020,21,6,12,32,...,False,False,False,False,False,True,False,False,False,False
4,fraud_Johnston-Casper,3.19,0,1126,0,2020,21,6,12,65,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
df['location'].nunique()

880

In [ ]:
df['state'] = df['location'].apply(lambda x: x.split(', ')[1])

# Print the DataFrame with the added 'Acronym' column
df['state'].unique()

array(['SC', 'UT', 'NY', 'FL', 'MI', 'CA', 'SD', 'PA', 'TX', 'KY', 'WY',
       'AL', 'LA', 'GA', 'CO', 'OH', 'WI', 'VT', 'AR', 'NJ', 'IA', 'MD',
       'MS', 'KS', 'IL', 'MO', 'ME', 'TN', 'DC', 'AZ', 'MT', 'MN', 'OK',
       'WA', 'WV', 'NM', 'MA', 'NE', 'VA', 'ID', 'OR', 'IN', 'NC', 'NH',
       'ND', 'CT', 'NV', 'HI', 'RI', 'AK'], dtype=object)

In [ ]:
region_mapping = {
    'North': {'ME', 'VT', 'NH', 'MA', 'RI', 'CT', 'NY', 'NJ', 'PA'},
    'Midwest': {'OH', 'MI', 'IN', 'WI', 'IL', 'MN', 'IA', 'MO', 'ND', 'SD', 'NE', 'KS'},
    'South': {'DE', 'MD', 'DC', 'VA', 'WV', 'NC', 'SC', 'GA', 'FL', 'KY', 'TN', 'AL', 'MS', 'AR', 'LA', 'TX', 'OK'},
    'West': {'MT', 'ID', 'WY', 'CO', 'NM', 'AZ', 'UT', 'NV', 'WA', 'OR', 'CA', 'AK', 'HI'}
}

# Create a function to map each state to its region
def map_region(state):
    for region, states in region_mapping.items():
        if state in states:
            return region
    return 'Unknown'

# Apply the mapping function to the DataFrame column
df['region'] = df['state'].apply(map_region)

# Print the DataFrame with regions
df.head()

,merchant,amt,gender,city_pop,is_fraud,Year,Day,Month,Hour,age,...,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,state,region
0,fraud_Kirlin and Sons,2.86,0,333497,0,2020,21,6,12,52,...,False,False,False,False,True,False,False,False,SC,South
1,fraud_Sporer-Keebler,29.84,1,302,0,2020,21,6,12,30,...,False,False,False,False,True,False,False,False,UT,West
2,"fraud_Swaniawski, Nitzsche and Welch",41.28,1,34496,0,2020,21,6,12,49,...,False,False,False,False,False,False,False,False,NY,North
3,fraud_Haley Group,60.05,0,54767,0,2020,21,6,12,32,...,False,False,False,True,False,False,False,False,FL,South
4,fraud_Johnston-Casper,3.19,0,1126,0,2020,21,6,12,65,...,False,False,False,False,False,False,False,True,MI,Midwest


In [ ]:
df = pd.get_dummies(df, columns=['region'])
df.head()

,merchant,amt,gender,city_pop,is_fraud,Year,Day,Month,Hour,age,...,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,state,region_Midwest,region_North,region_South,region_West
0,fraud_Kirlin and Sons,2.86,0,333497,0,2020,21,6,12,52,...,False,True,False,False,False,SC,False,False,True,False
1,fraud_Sporer-Keebler,29.84,1,302,0,2020,21,6,12,30,...,False,True,False,False,False,UT,False,False,False,True
2,"fraud_Swaniawski, Nitzsche and Welch",41.28,1,34496,0,2020,21,6,12,49,...,False,False,False,False,False,NY,False,True,False,False
3,fraud_Haley Group,60.05,0,54767,0,2020,21,6,12,32,...,True,False,False,False,False,FL,False,False,True,False
4,fraud_Johnston-Casper,3.19,0,1126,0,2020,21,6,12,65,...,False,False,False,False,True,MI,True,False,False,False


In [ ]:
df.columns

Index(['merchant', 'amt', 'gender', 'city_pop', 'is_fraud', 'Year', 'Day',
       'Month', 'Hour', 'age', 'location', 'lat_long', 'merchant_lat_long',
       'job_Agriculture', 'job_Business', 'job_Creative', 'job_Education',
       'job_Engineering_Technical', 'job_Healthcare', 'job_Legal', 'job_Media',
       'job_Social_Services', 'category_entertainment', 'category_food_dining',
       'category_gas_transport', 'category_grocery_net',
       'category_grocery_pos', 'category_health_fitness', 'category_home',
       'category_kids_pets', 'category_misc_net', 'category_misc_pos',
       'category_personal_care', 'category_shopping_net',
       'category_shopping_pos', 'category_travel', 'state', 'region_Midwest',
       'region_North', 'region_South', 'region_West'],
      dtype='object')

In [ ]:
#Year is 2020 for all values
df.drop(['state','Year','location'], axis= 1, inplace=True)
df.head()

,merchant,amt,gender,city_pop,is_fraud,Day,Month,Hour,age,lat_long,...,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,region_Midwest,region_North,region_South,region_West
0,fraud_Kirlin and Sons,2.86,0,333497,0,21,6,12,52,"33.9659, -80.9355",...,False,False,True,False,False,False,False,False,True,False
1,fraud_Sporer-Keebler,29.84,1,302,0,21,6,12,30,"40.3207, -110.436",...,False,False,True,False,False,False,False,False,False,True
2,"fraud_Swaniawski, Nitzsche and Welch",41.28,1,34496,0,21,6,12,49,"40.6729, -73.5365",...,False,False,False,False,False,False,False,True,False,False
3,fraud_Haley Group,60.05,0,54767,0,21,6,12,32,"28.5697, -80.8191",...,False,True,False,False,False,False,False,False,True,False
4,fraud_Johnston-Casper,3.19,0,1126,0,21,6,12,65,"44.2529, -85.017",...,False,False,False,False,False,True,True,False,False,False


In [ ]:
df.columns

Index(['merchant', 'amt', 'gender', 'city_pop', 'is_fraud', 'Day', 'Month',
       'Hour', 'age', 'lat_long', 'merchant_lat_long', 'job_Agriculture',
       'job_Business', 'job_Creative', 'job_Education',
       'job_Engineering_Technical', 'job_Healthcare', 'job_Legal', 'job_Media',
       'job_Social_Services', 'category_entertainment', 'category_food_dining',
       'category_gas_transport', 'category_grocery_net',
       'category_grocery_pos', 'category_health_fitness', 'category_home',
       'category_kids_pets', 'category_misc_net', 'category_misc_pos',
       'category_personal_care', 'category_shopping_net',
       'category_shopping_pos', 'category_travel', 'region_Midwest',
       'region_North', 'region_South', 'region_West'],
      dtype='object')

In [ ]:
for col in df.columns:
    if col.startswith('job_') or col.startswith('region_') or col.startswith('category_'):
        df[col] = df[col].astype(int)

In [ ]:
df.head()

,merchant,amt,gender,city_pop,is_fraud,Day,Month,Hour,age,lat_long,...,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,region_Midwest,region_North,region_South,region_West
0,fraud_Kirlin and Sons,2.86,0,333497,0,21,6,12,52,"33.9659, -80.9355",...,0,0,1,0,0,0,0,0,1,0
1,fraud_Sporer-Keebler,29.84,1,302,0,21,6,12,30,"40.3207, -110.436",...,0,0,1,0,0,0,0,0,0,1
2,"fraud_Swaniawski, Nitzsche and Welch",41.28,1,34496,0,21,6,12,49,"40.6729, -73.5365",...,0,0,0,0,0,0,0,1,0,0
3,fraud_Haley Group,60.05,0,54767,0,21,6,12,32,"28.5697, -80.8191",...,0,1,0,0,0,0,0,0,1,0
4,fraud_Johnston-Casper,3.19,0,1126,0,21,6,12,65,"44.2529, -85.017",...,0,0,0,0,0,1,1,0,0,0


In [ ]:
from geopy.distance import geodesic
# Define a function to compute the distance between two coordinates
def compute_distance(row):
    return geodesic(row['lat_long'], row['merchant_lat_long']).kilometers

# Apply the function to each row to compute the distance
df['distance'] = df.apply(compute_distance, axis=1)

In [ ]:
df.columns

Index(['merchant', 'amt', 'gender', 'city_pop', 'is_fraud', 'Day', 'Month',
       'Hour', 'age', 'lat_long', 'merchant_lat_long', 'job_Agriculture',
       'job_Business', 'job_Creative', 'job_Education',
       'job_Engineering_Technical', 'job_Healthcare', 'job_Legal', 'job_Media',
       'job_Social_Services', 'category_entertainment', 'category_food_dining',
       'category_gas_transport', 'category_grocery_net',
       'category_grocery_pos', 'category_health_fitness', 'category_home',
       'category_kids_pets', 'category_misc_net', 'category_misc_pos',
       'category_personal_care', 'category_shopping_net',
       'category_shopping_pos', 'category_travel', 'region_Midwest',
       'region_North', 'region_South', 'region_West', 'distance_to_center',
       'distance'],
      dtype='object')

In [ ]:
df.drop(['distance_to_center','lat_long','merchant_lat_long'], axis= 1, inplace=True)

In [ ]:
df.drop(['merchant'],axis=1,inplace=True)

In [ ]:
df.head()

,amt,gender,city_pop,is_fraud,Day,Month,Hour,age,job_Agriculture,job_Business,...,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,region_Midwest,region_North,region_South,region_West,distance
0,2.86,0,333497,0,21,6,12,52,0,0,...,0,1,0,0,0,0,0,1,0,24.613746
1,29.84,1,302,0,21,6,12,30,0,1,...,0,1,0,0,0,0,0,0,1,104.834043
2,41.28,1,34496,0,21,6,12,49,0,0,...,0,0,0,0,0,0,1,0,0,59.204796
3,60.05,0,54767,0,21,6,12,32,0,0,...,1,0,0,0,0,0,0,1,0,27.615117
4,3.19,0,1126,0,21,6,12,65,0,0,...,0,0,0,0,1,1,0,0,0,104.423175


In [ ]:
# Function to get the weekday name corresponding to the given day and month
from datetime import datetime

def get_weekday(day, month):
    date_str = f'2020-{int(month)}-{int(day)}'  # Construct a date string in the format 'YYYY-MM-DD'
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')  # Parse the date string to a datetime object
    return date_obj.strftime('%A')  # Get the weekday name

# Apply the function to create a new column 'weekday'
df['weekday'] = df.apply(lambda row: get_weekday(row['Day'], row['Month']), axis=1)

In [ ]:
df = pd.get_dummies(df, columns=['weekday'])

for col in df.columns:
    if col.startswith('weekday_'):
        df[col] = df[col].astype(int)

df.head()

In [ ]:
df.head()

,amt,gender,city_pop,is_fraud,Day,Month,Hour,age,job_Agriculture,job_Business,...,region_South,region_West,distance,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,2.86,0,333497,0,21,6,12,52,0,0,...,1,0,24.613746,0,0,0,1,0,0,0
1,29.84,1,302,0,21,6,12,30,0,1,...,0,1,104.834043,0,0,0,1,0,0,0
2,41.28,1,34496,0,21,6,12,49,0,0,...,0,0,59.204796,0,0,0,1,0,0,0
3,60.05,0,54767,0,21,6,12,32,0,0,...,1,0,27.615117,0,0,0,1,0,0,0
4,3.19,0,1126,0,21,6,12,65,0,0,...,0,0,104.423175,0,0,0,1,0,0,0


In [ ]:
df.drop(['Day'],axis=1,inplace = True)

In [ ]:
df.columns

Index(['amt', 'gender', 'city_pop', 'is_fraud', 'Month', 'Hour', 'age',
       'job_Agriculture', 'job_Business', 'job_Creative', 'job_Education',
       'job_Engineering_Technical', 'job_Healthcare', 'job_Legal', 'job_Media',
       'job_Social_Services', 'category_entertainment', 'category_food_dining',
       'category_gas_transport', 'category_grocery_net',
       'category_grocery_pos', 'category_health_fitness', 'category_home',
       'category_kids_pets', 'category_misc_net', 'category_misc_pos',
       'category_personal_care', 'category_shopping_net',
       'category_shopping_pos', 'category_travel', 'region_Midwest',
       'region_North', 'region_South', 'region_West', 'distance',
       'weekday_Friday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday'],
      dtype='object')

In [ ]:
# scale age, amt, city_pop , Month , Hour ,distance
from sklearn.preprocessing import StandardScaler

# Define columns to standardize
columns_to_standardize = ['age', 'amt', 'city_pop', 'Month', 'Hour', 'distance']

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the selected columns
df_scaled = df.copy()
df_scaled[columns_to_standardize] = scaler.fit_transform(df_scaled[columns_to_standardize])

# Display the DataFrame with scaled features
df_scaled.head()

,amt,gender,city_pop,is_fraud,Month,Hour,age,job_Agriculture,job_Business,job_Creative,...,region_South,region_West,distance,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,-0.424463,0,0.816521,0,-1.773597,-0.118789,0.319687,0,0,0,...,1,0,-1.769819,0,0,0,1,0,0,0
1,-0.252337,1,-0.292685,0,-1.773597,-0.118789,-0.941585,0,1,0,...,0,1,0.987574,0,0,0,1,0,0,0
2,-0.179353,1,-0.178853,0,-1.773597,-0.118789,0.147695,0,0,0,...,0,0,-0.580829,0,0,0,1,0,0,0
3,-0.059605,0,-0.111371,0,-1.773597,-0.118789,-0.826924,0,0,1,...,1,0,-1.666654,0,0,0,1,0,0,0
4,-0.422358,0,-0.289942,0,-1.773597,-0.118789,1.064983,0,0,1,...,0,0,0.973451,0,0,0,1,0,0,0


In [ ]:
import pickle

# Save the scaler object as a pickle file
with open('ogdata2.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
#when de-standardizing in the future
import pickle

# Load the scaler object
with open('ogdata2.pkl', 'rb') as file:
    scaler_loaded = pickle.load(file)

df_destandardized = df_scaled.copy()
df_destandardized[columns_to_standardize] = scaler_loaded.inverse_transform(df_destandardized[columns_to_standardize])

# Display the DataFrame with destandardized features
df_destandardized.head()

,amt,gender,city_pop,is_fraud,Month,Hour,age,job_Agriculture,job_Business,job_Creative,...,region_South,region_West,distance,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,2.86,0,333497.0,0,6.0,12.0,52.0,0,0,0,...,1,0,24.613746,0,0,0,1,0,0,0
1,29.84,1,302.0,0,6.0,12.0,30.0,0,1,0,...,0,1,104.834043,0,0,0,1,0,0,0
2,41.28,1,34496.0,0,6.0,12.0,49.0,0,0,0,...,0,0,59.204796,0,0,0,1,0,0,0
3,60.05,0,54767.0,0,6.0,12.0,32.0,0,0,1,...,1,0,27.615117,0,0,0,1,0,0,0
4,3.19,0,1126.0,0,6.0,12.0,65.0,0,0,1,...,0,0,104.423175,0,0,0,1,0,0,0


day , month , hour  
--> age(Scale)
--> amt(Scale)
--> lat_long & merchant_lat_long --> distance then scale
--> city_pop (Scale)
--> drop day and replaced with weekday
--> month , hour (Scale)

In [ ]:
df_scaled.head()

,amt,gender,city_pop,is_fraud,Month,Hour,age,job_Agriculture,job_Business,job_Creative,...,region_South,region_West,distance,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,-0.424463,0,0.816521,0,-1.773597,-0.118789,0.319687,0,0,0,...,1,0,-1.769819,0,0,0,1,0,0,0
1,-0.252337,1,-0.292685,0,-1.773597,-0.118789,-0.941585,0,1,0,...,0,1,0.987574,0,0,0,1,0,0,0
2,-0.179353,1,-0.178853,0,-1.773597,-0.118789,0.147695,0,0,0,...,0,0,-0.580829,0,0,0,1,0,0,0
3,-0.059605,0,-0.111371,0,-1.773597,-0.118789,-0.826924,0,0,1,...,1,0,-1.666654,0,0,0,1,0,0,0
4,-0.422358,0,-0.289942,0,-1.773597,-0.118789,1.064983,0,0,1,...,0,0,0.973451,0,0,0,1,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

# Define features and target variable
X = df_scaled.drop('is_fraud', axis=1)
y = df_scaled['is_fraud']

# Split the data into training and testing sets with 80-20 ratio, preserving the class ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Display the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (444575, 41) (444575,)
Testing set shape: (111144, 41) (111144,)


In [ ]:
# Concatenate X_train with y_train
train = pd.concat([X_train, y_train], axis=1)

# Concatenate X_test with y_test
test = pd.concat([X_test, y_test], axis=1)

# Display the shapes of the concatenated datasets
print("Training data shape:", train.shape)
print("Testing data shape:", test.shape)

Training data shape: (444575, 42)
Testing data shape: (111144, 42)


In [ ]:
test['is_fraud'].value_counts()

is_fraud
0    110715
1       429
Name: count, dtype: int64

In [ ]:
df.head()

,amt,gender,city_pop,is_fraud,Month,Hour,age,job_Agriculture,job_Business,job_Creative,...,region_South,region_West,distance,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,2.86,0,333497,0,6,12,52,0,0,0,...,1,0,24.613746,0,0,0,1,0,0,0
1,29.84,1,302,0,6,12,30,0,1,0,...,0,1,104.834043,0,0,0,1,0,0,0
2,41.28,1,34496,0,6,12,49,0,0,0,...,0,0,59.204796,0,0,0,1,0,0,0
3,60.05,0,54767,0,6,12,32,0,0,1,...,1,0,27.615117,0,0,0,1,0,0,0
4,3.19,0,1126,0,6,12,65,0,0,1,...,0,0,104.423175,0,0,0,1,0,0,0


In [ ]:
# Define file paths
train_file = "train.csv"
test_file = "test.csv"
df_file = "ogcc.csv"
df_scaled_file = "scaledcc.csv"

# Save DataFrames to CSV files
train_data.to_csv(train_file, index=False)
test_data.to_csv(test_file, index=False)
df.to_csv(df_file, index=False)
df_scaled.to_csv(df_scaled_file, index=False)

print("DataFrames saved as CSV files successfully.")

DataFrames saved as CSV files successfully.
